# Feature engineering

* Feature selection
* Feature transformation

In [168]:
import pandas as pd

In [169]:
df = pd.read_csv('../input/cleaned_data.csv')

df

,building_id,building_type,primary_property_type,property_name,neighborhood,list_of_all_property_use_types,largest_property_use_type,second_largest_property_use_type,default_data,compliance_status,...,steam_use,electricity_kwh,electricity,natural_gas_therms,natural_gas,emissions_target,emissions_intensity,is_agregation,coords,distance_to_center
0,23163,NonResidential,Medical Office,LESCHI CENTER,downtown,Medical Office,Medical Office,NaN,Yes,Compliant,...,0.000,5.272270e+05,1.798974e+06,0.000000,0.000000e+00,12.540,0.110,False,"(47.59703102, -122.3168033)",1.51
1,23848,NonResidential,Restaurant\n,SPAGHETTI FACTORY,downtown,"Office, Restaurant",Restaurant,Office,No,Compliant,...,0.000,5.316360e+05,1.814019e+06,20028.000000,2.002751e+06,119.010,2.760,False,"(47.61522339, -122.354154)",1.95
2,21471,NonResidential,Other,EITEL BUILDING (2ND AND PIKE BUILDING),downtown,"Other, Personal Services (Health/Beauty, Dry C...",Other,Retail Store,No,Compliant,...,0.000,1.115070e+05,3.804780e+05,335.000000,3.345700e+04,4.430,0.060,False,"(47.60940552, -122.33934021)",0.67
3,21237,NonResidential,Retail Store,STAPLES,northwest,"Parking, Retail Store",Retail Store,Parking,Yes,Compliant,...,0.000,2.466200e+05,8.415010e+05,5507.000000,5.506860e+05,35.110,1.330,False,"(47.72569066, -122.3432661)",13.34
4,22138,NonResidential,Worship Facility,BRIGHTON PRESBYTERIAN CHURCH,southeast,NaN,NaN,NaN,No,Compliant,...,0.000,2.696300e+04,9.200000e+04,0.000000,0.000000e+00,0.640,0.010,False,"(47.54179702, -122.2703046)",8.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1692,775,NonResidential,Large Office,Columbia Center - 2015,downtown,"Office, Parking",Office,Parking,False,Compliant,...,0.000,2.617277e+07,8.930337e+07,44473.816405,4.447364e+06,858.760,0.340,True,"(47.60454, -122.33072)",0.19
1693,753,NonResidential,Other,WestinBuilding,downtown,"Data Center, Office",Data Center,Office,False,Compliant,...,0.000,8.195296e+07,2.796294e+08,1511.360107,1.511555e+05,1957.390,3.135,True,"(47.61438, -122.33871)",1.06
1694,752,NonResidential,Large Office,ACTIVE VOICE,downtown,"Bar/Nightclub, Office, Personal Services (Heal...",Office,Bar/Nightclub,False,Compliant,...,2806907.625,1.945288e+06,6.637447e+06,13514.160155,1.351432e+06,334.705,1.905,True,"(47.61481, -122.33969)",1.14
1695,857,NonResidential,Warehouse,3480 W Marginal Way SW,southwest,"Non-Refrigerated Warehouse, Office",Non-Refrigerated Warehouse,Office,False,Compliant,...,0.000,2.852932e+05,9.734410e+05,1833.715027,1.833580e+05,16.525,0.095,True,"(47.57311, -122.3601)",4.22


## Correlation between the features

We must remove variables that are too much correlated to our targets :

* `emissions_target`
* `site_energy_use_target`

In [170]:
corr = df.corr()
corr.style.background_gradient(cmap='RdBu', vmin = -1, vmax = 1)

,building_id,latitude,longitude,zip_code,data_year,council_district_code,year_built,number_of_buildings,number_of_floors,property_gfa_total,property_gfa_parking,property_gfa_building,largest_property_use_type_gfa,second_largest_property_use_type_gfa,energystar_score,site_eui,site_euiwn,source_eui,source_euiwn,site_energy_use_target,site_energy_use_wn,steam_use,electricity_kwh,electricity,natural_gas_therms,natural_gas,emissions_target,emissions_intensity,is_agregation,distance_to_center
building_id,1.000000,0.034450,0.088578,0.136511,0.257565,-0.160903,-0.004553,0.055623,-0.252416,-0.146869,-0.207632,-0.122696,-0.109795,-0.229747,-0.048141,-0.003153,0.001637,-0.045042,-0.044600,-0.084608,-0.175258,-0.019759,-0.100271,-0.100274,-0.060701,-0.060701,-0.071000,0.085183,-0.262344,0.157171
latitude,0.034450,1.000000,-0.100032,-0.000151,0.015274,0.525609,0.066302,0.016819,-0.010693,0.005713,0.034534,0.000725,0.005768,-0.015685,0.049412,0.098068,0.094407,0.092963,0.090819,0.009873,-0.008045,-0.005622,0.011912,0.011912,0.001463,0.001463,0.000170,0.043333,-0.023232,0.356204
longitude,0.088578,-0.100032,1.000000,-0.011034,0.051297,-0.338463,0.016426,0.027111,-0.013214,0.018620,-0.037126,0.025261,0.027294,0.011776,-0.005151,-0.018650,-0.020924,-0.021792,-0.024617,0.028324,0.017067,0.021568,0.018950,0.018950,0.028446,0.028446,0.034667,0.005995,-0.047279,0.045347
zip_code,0.136511,-0.000151,-0.011034,1.000000,0.134341,-0.196102,0.101844,-0.014298,-0.145180,-0.059590,-0.100601,-0.047274,-0.040023,-0.075435,0.010473,-0.064862,-0.071852,-0.059324,-0.067312,-0.050810,-0.061288,-0.052293,-0.049397,-0.049398,-0.023241,-0.023241,-0.054101,-0.066441,-0.123636,0.198540
data_year,0.257565,0.015274,0.051297,0.134341,1.000000,-0.040442,0.105034,0.099735,0.003414,0.094981,-0.024462,0.103631,0.105118,0.059237,-0.000617,0.053724,0.038466,0.049154,0.035714,0.111344,0.038361,0.026254,0.103512,0.103510,0.068468,0.068468,0.080428,0.087345,-0.241424,0.053823
council_district_code,-0.160903,0.525609,-0.338463,-0.196102,-0.040442,1.000000,-0.051421,-0.016575,0.274528,0.108793,0.200871,0.083686,0.069642,0.172182,0.083467,0.089137,0.080249,0.113321,0.109846,0.052812,0.073583,0.009346,0.077812,0.077814,-0.005874,-0.005874,0.016301,0.000400,-0.011163,-0.220561
year_built,-0.004553,0.066302,0.016426,0.101844,0.105034,-0.051421,1.000000,-0.035497,0.095455,0.105372,0.289564,0.066522,0.071967,0.252805,-0.031097,0.127640,0.119724,0.162819,0.159387,0.070705,0.138741,0.001126,0.080385,0.080387,0.064005,0.064005,0.059041,-0.010144,-0.048813,0.239841
number_of_buildings,0.055623,0.016819,0.027111,-0.014298,0.099735,-0.016575,-0.035497,1.000000,-0.023015,0.775345,0.001809,0.815135,0.830086,0.175061,-0.053791,0.020767,-0.008380,0.020628,-0.010056,0.723498,0.079243,0.396027,0.774696,0.774691,0.062941,0.062941,0.411479,0.013595,-0.105332,0.020822
number_of_floors,-0.252416,-0.010693,-0.013214,-0.145180,0.003414,0.274528,0.095455,-0.023015,1.000000,0.405350,0.505516,0.348965,0.334939,0.493506,0.115653,0.025231,0.011125,0.063827,0.054739,0.220255,0.307664,0.097412,0.281331,0.281335,0.035172,0.035172,0.131963,-0.049174,0.021720,-0.312387
property_gfa_total,-0.146869,0.005713,0.018620,-0.059590,0.094981,0.108793,0.105372,0.775345,0.405350,1.000000,0.402231,0.990144,0.978178,0.801810,0.122794,0.039002,0.004660,0.056666,0.024729,0.810282,0.378890,0.464483,0.866648,0.866647,0.163843,0.163843,0.534774,-0.005494,-0.086332,-0.142021


### Feature creation and engineering

In [171]:
df.head()

,building_id,building_type,primary_property_type,property_name,neighborhood,list_of_all_property_use_types,largest_property_use_type,second_largest_property_use_type,default_data,compliance_status,...,steam_use,electricity_kwh,electricity,natural_gas_therms,natural_gas,emissions_target,emissions_intensity,is_agregation,coords,distance_to_center
0,23163,NonResidential,Medical Office,LESCHI CENTER,downtown,Medical Office,Medical Office,NaN,Yes,Compliant,...,0.0,527227.0,1798974.0,0.0,0.0,12.54,0.11,False,"(47.59703102, -122.3168033)",1.51
1,23848,NonResidential,Restaurant\n,SPAGHETTI FACTORY,downtown,"Office, Restaurant",Restaurant,Office,No,Compliant,...,0.0,531636.0,1814019.0,20028.0,2002751.0,119.01,2.76,False,"(47.61522339, -122.354154)",1.95
2,21471,NonResidential,Other,EITEL BUILDING (2ND AND PIKE BUILDING),downtown,"Other, Personal Services (Health/Beauty, Dry C...",Other,Retail Store,No,Compliant,...,0.0,111507.0,380478.0,335.0,33457.0,4.43,0.06,False,"(47.60940552, -122.33934021)",0.67
3,21237,NonResidential,Retail Store,STAPLES,northwest,"Parking, Retail Store",Retail Store,Parking,Yes,Compliant,...,0.0,246620.0,841501.0,5507.0,550686.0,35.11,1.33,False,"(47.72569066, -122.3432661)",13.34
4,22138,NonResidential,Worship Facility,BRIGHTON PRESBYTERIAN CHURCH,southeast,NaN,NaN,NaN,No,Compliant,...,0.0,26963.0,92000.0,0.0,0.0,0.64,0.01,False,"(47.54179702, -122.2703046)",8.51


In [172]:
df['surface_per_floor'] = df['property_gfa_building'] / df['number_of_floors']
df['surface_per_building'] = df['property_gfa_building'] / df['number_of_buildings']
df['age'] = 2022 - df['year_built']
df['have_parking'] = df['property_gfa_parking'] > 0

In [173]:
df['building_primary_type'] = df['building_type'] + ' ' + df['primary_property_type']

In [174]:
# https://github.com/JamesIgoe/GoogleFitAnalysis/blob/master/Analysis.ipynb

def corrFilter(x: pd.DataFrame, bound: float):
    xCorr = x.corr()
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    return xFiltered

def corrFilterFlattened(x: pd.DataFrame, bound: float):
    xFiltered = corrFilter(x, bound)
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    return xFlattened

def filterForLabels(df: pd.DataFrame, label):
    df = df.sort_index()
    try:
        sideLeft = df[label,]
    except:
        sideLeft = pd.DataFrame()

    try:
        sideRight = df[:,label]
    except:
        sideRight = pd.DataFrame()

    if sideLeft.empty and sideRight.empty:
        return pd.DataFrame()
    elif sideLeft.empty:        
        concat = sideRight.to_frame(name='correlation').rename_axis('variable').reset_index(level=0)
        return concat
    elif sideRight.empty:
        concat = sideLeft.to_frame(name='correlation').rename_axis('variable').reset_index(level=0)
        return concat
    else:
        concat = pd.concat([sideLeft,sideRight], axis=1)
        concat['correlation'] = concat[0].fillna(0) + concat[1].fillna(0)
        concat.drop(columns=[0,1], inplace=True)

        return concat.rename_axis('variable').reset_index(level=0)

In [175]:
corr_df = corrFilterFlattened(df, 0.7)


filterForLabels(corr_df, 'emissions_target')

,variable,correlation
0,natural_gas,0.737067
1,natural_gas_therms,0.737067
2,site_energy_use_target,0.861910
3,site_energy_use_wn,0.859331


In [176]:
filterForLabels(corr_df, 'site_energy_use_target')

,variable,correlation
0,electricity_kwh,0.956665
1,property_gfa_building,0.824156
2,property_gfa_total,0.810282
3,site_energy_use_wn,0.707030
4,electricity,0.956665
5,emissions_target,0.861910
6,largest_property_use_type_gfa,0.844702
7,number_of_buildings,0.723498


In [177]:
variables_to_remove = filterForLabels(corr_df, 'site_energy_use_target')['variable'].tolist()
df = df.drop(variables_to_remove, axis='columns')

In [178]:
# remove useless variables

df = df.drop([
        'building_id',
        'property_name',
        'default_data',
        'compliance_status',
        'site_eui',
        'site_euiwn',
        'source_euiwn',
        'source_eui',
        'steam_use',
        'emissions_intensity',
        'natural_gas',
        'natural_gas_therms',
        'second_largest_property_use_type_gfa',
        'second_largest_property_use_type',
        'number_of_floors',
        'property_gfa_parking',
        'latitude',
        'longitude',
        'address',
        'data_year',
        'is_agregation',
        'zip_code',
        'year_built',
        ],axis='columns')

In [179]:
corr = df.corr()
corr.style.background_gradient(cmap='RdBu', vmin = -1, vmax = 1)

,council_district_code,energystar_score,site_energy_use_target,distance_to_center,surface_per_floor,surface_per_building,age,have_parking
council_district_code,1.000000,0.083467,0.052812,-0.220561,-0.150258,0.163236,0.051421,0.263068
energystar_score,0.083467,1.000000,-0.078893,-0.041238,0.028210,0.126068,0.031097,0.008580
site_energy_use_target,0.052812,-0.078893,1.000000,-0.080841,0.300414,0.318270,-0.070705,0.083615
distance_to_center,-0.220561,-0.041238,-0.080841,1.000000,0.120340,-0.225110,-0.239841,-0.213792
surface_per_floor,-0.150258,0.028210,0.300414,0.120340,1.000000,0.264930,-0.088301,-0.097634
surface_per_building,0.163236,0.126068,0.318270,-0.225110,0.264930,1.000000,-0.168501,0.220711
age,0.051421,0.031097,-0.070705,-0.239841,-0.088301,-0.168501,1.000000,-0.352081
have_parking,0.263068,0.008580,0.083615,-0.213792,-0.097634,0.220711,-0.352081,1.000000


In [180]:
print('Done')

Done
